In [1]:
import pandas as pd
import requests
import re
import time

In [2]:
hdb = pd.read_csv('hdb-property-information.csv')
hdb.head()

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,...,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
0,1,BEACH RD,16,1970,Y,Y,N,N,N,N,...,138,1,2,0,0,0,0,0,0,0
1,1,BEDOK STH AVE 1,14,1975,Y,N,N,Y,N,N,...,204,0,2,0,0,0,0,0,0,0
2,1,CANTONMENT RD,2,2010,N,Y,N,N,N,N,...,0,0,0,0,0,0,0,0,0,0
3,1,CHAI CHEE RD,15,1982,Y,N,N,N,N,N,...,0,10,92,0,0,0,0,0,0,0
4,1,CHANGI VILLAGE RD,4,1975,Y,Y,N,N,N,N,...,54,0,1,0,0,0,0,0,0,0


In [3]:
# sample
data = requests.get('https://developers.onemap.sg/commonapi/search?searchVal=9c+boon+tiong+road&returnGeom=Y&getAddrDetails=Y&pageNum=1')

In [4]:
data.content

b'{"found":1,"totalNumPages":1,"pageNum":1,"results":[{"SEARCHVAL":"TIONG BAHRU VIEW","BLK_NO":"9C","ROAD_NAME":"BOON TIONG ROAD","BUILDING":"TIONG BAHRU VIEW","ADDRESS":"9C BOON TIONG ROAD TIONG BAHRU VIEW SINGAPORE 164009","POSTAL":"164009","X":"27533.064019999998","Y":"29918.253210000003","LATITUDE":"1.28684456","LONGITUDE":"103.829123","LONGTITUDE":"103.829123"}]}'

In [5]:
address = hdb.loc[:, ['blk_no', 'street']]
address['query'] = address['blk_no'].astype(str) + '+' + address['street'].astype(str)
address['query'] = [re.sub(' ', '+', query) for query in address['query']]
address.head()

,blk_no,street,query
0,1,BEACH RD,1+BEACH+RD
1,1,BEDOK STH AVE 1,1+BEDOK+STH+AVE+1
2,1,CANTONMENT RD,1+CANTONMENT+RD
3,1,CHAI CHEE RD,1+CHAI+CHEE+RD
4,1,CHANGI VILLAGE RD,1+CHANGI+VILLAGE+RD


In [6]:
address_dict = address.to_dict('records')

In [9]:
for i in range(0, len(address_dict)):
    if i % 100 == 0:
        print('at ' + str(i))

    try:
        record = address_dict[i]
        query = 'https://developers.onemap.sg/commonapi/search?searchVal={0}&returnGeom=Y&getAddrDetails=Y&pageNum=1'.format(record['query'])
        resp = requests.get(query)
        if resp.status_code == 200:
            content = resp.json()
            if content['found'] > 0:
                record['coordinates'] = [content['results'][0]['LATITUDE'], content['results'][0]['LONGTITUDE']]
                record['found'] = content['found']
                record['full'] = content['results']
                record['ok'] = 1
                continue
        
        print('sth failed at record ' + str(i))
        record['ok'] = 0

    except Exception as e:
        print('exception ' + str(e) + 'at record ' + str(i))
        record['ok'] = 0
    
    time.sleep(0.08)

search?searchVal=438B+SENGKANG+WEST+AVE&returnGeom=Y&getAddrDetails=Y&pageNum=1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000025108934C18>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))at record 6689
exception HTTPSConnectionPool(host='developers.onemap.sg', port=443): Max retries exceeded with url: /commonapi/search?searchVal=438C+BT+BATOK+WEST+AVE+8&returnGeom=Y&getAddrDetails=Y&pageNum=1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000251089343C8>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))at record 6690
exception HTTPSConnectionPool(host='developers.onemap.sg', port=443): Max retries exceeded with url: /commonapi/search?searchVal=439+ANG+MO+KIO+AVE+10&returnGeom=Y&getAddrDetails=Y&pageNum=1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000025108934780>: Failed to establish a new connection: [Er

In [10]:
address_dict[0]

{'blk_no': '1',
 'street': 'BEACH RD',
 'query': '1+BEACH+RD',
 'ok': 1,
 'coordinates': ['1.303671351', '103.8644787'],
 'found': 3,
 'full': [{'SEARCHVAL': 'BEACH ROAD GARDENS',
   'BLK_NO': '1',
   'ROAD_NAME': 'BEACH ROAD',
   'BUILDING': 'BEACH ROAD GARDENS',
   'ADDRESS': '1 BEACH ROAD BEACH ROAD GARDENS SINGAPORE 190001',
   'POSTAL': '190001',
   'X': '31467.832739999998',
   'Y': '31778.894',
   'LATITUDE': '1.303671351',
   'LONGITUDE': '103.8644787',
   'LONGTITUDE': '103.8644787'},
  {'SEARCHVAL': 'RAFFLES HOTEL',
   'BLK_NO': '1',
   'ROAD_NAME': 'BEACH ROAD',
   'BUILDING': 'RAFFLES HOTEL',
   'ADDRESS': '1 BEACH ROAD RAFFLES HOTEL SINGAPORE 189673',
   'POSTAL': '189673',
   'X': '30309.26725',
   'Y': '30830.8196',
   'LATITUDE': '1.295097408',
   'LONGITUDE': '103.8540683',
   'LONGTITUDE': '103.8540683'},
  {'SEARCHVAL': 'RAFFLES HOTEL SINGAPORE',
   'BLK_NO': '1',
   'ROAD_NAME': 'BEACH ROAD',
   'BUILDING': 'RAFFLES HOTEL SINGAPORE',
   'ADDRESS': '1 BEACH ROAD RAFF

In [21]:
import pickle as pkl
pkl.dump(address_dict, open('address_dict_v1.pkl', 'wb'))

In [13]:
add_df = pd.DataFrame(address_dict)
add_df.head()

,blk_no,coordinates,found,full,ok,query,street
0,1,"[1.303671351, 103.8644787]",3.0,"[{'SEARCHVAL': 'BEACH ROAD GARDENS', 'BLK_NO':...",1,1+BEACH+RD,BEACH RD
1,1,"[1.320852087, 103.93372109999999]",2.0,"[{'SEARCHVAL': 'HDB-BEDOK', 'BLK_NO': '1', 'RO...",1,1+BEDOK+STH+AVE+1,BEDOK STH AVE 1
2,1,"[1.277034744, 103.841261]",3.0,"[{'SEARCHVAL': 'THE PINNACLE@DUXTON', 'BLK_NO'...",1,1+CANTONMENT+RD,CANTONMENT RD
3,1,"[1.327968792, 103.92271600000001]",1.0,"[{'SEARCHVAL': 'PING YI GARDENS', 'BLK_NO': '1...",1,1+CHAI+CHEE+RD,CHAI CHEE RD
4,1,"[1.388546562, 103.98780450000001]",2.0,"[{'SEARCHVAL': 'HDB-CHANGI VILLAGE ROV', 'BLK_...",1,1+CHANGI+VILLAGE+RD,CHANGI VILLAGE RD


In [16]:
error = list(add_df.loc[add_df['ok'] == 0, :].index.values)

In [19]:
not_found = []
for i in error:
    if i % 10 == 0:
        print('at ' + str(i))

    try:
        record = address_dict[i]
        query = 'https://developers.onemap.sg/commonapi/search?searchVal={0}&returnGeom=Y&getAddrDetails=Y&pageNum=1'.format(record['query'])
        resp = requests.get(query)
        if resp.status_code == 200:
            content = resp.json()
            if content['found'] > 0:
                record['coordinates'] = [content['results'][0]['LATITUDE'], content['results'][0]['LONGTITUDE']]
                record['found'] = content['found']
                record['full'] = content['results']
                record['ok'] = 1
                continue
        
        not_found.append(i)
        print('sth failed at record ' + str(i))
        record['ok'] = 0

    except Exception as e:
        print('exception ' + str(e) + 'at record ' + str(i))
        record['ok'] = 0
    
    time.sleep(0.08)

exception HTTPSConnectionPool(host='developers.onemap.sg', port=443): Max retries exceeded with url: /commonapi/search?searchVal=1+ST.+GEORGE'S+RD&returnGeom=Y&getAddrDetails=Y&pageNum=1 (Caused by SSLError(SSLError("bad handshake: SysCallError(10060, 'WSAETIMEDOUT')")))at record 25
sth failed at record 66
sth failed at record 468
sth failed at record 843
sth failed at record 1178
sth failed at record 1431
sth failed at record 1635
sth failed at record 1837
sth failed at record 2048
sth failed at record 2217
sth failed at record 2229
at 2420
sth failed at record 2420
at 2520
sth failed at record 2546
sth failed at record 2593
sth failed at record 2862
sth failed at record 3103
sth failed at record 3321
sth failed at record 3514
sth failed at record 3657
sth failed at record 3665
at 3840
sth failed at record 3840
sth failed at record 3848
at 3850
sth failed at record 3850
at 4020
sth failed at record 4542
at 5760
at 5770
at 5780
at 5790
at 5990
sth failed at record 5990
sth failed at re

In [20]:
add_df2 = pd.DataFrame(address_dict)
len(add_df2.loc[add_df2['ok'] == 0, :])

31

In [31]:
error = list(add_df2.loc[add_df2['ok'] == 0, :].index.values)

In [24]:
add_df2.loc[add_df2['ok'] == 0, :]

,blk_no,coordinates,found,full,ok,query,street
25,1,NaN,NaN,NaN,0,1+ST.+GEORGE'S+RD,ST. GEORGE'S RD
66,10,NaN,NaN,NaN,0,10+ST.+GEORGE'S+RD,ST. GEORGE'S RD
468,11,NaN,NaN,NaN,0,11+ST.+GEORGE'S+RD,ST. GEORGE'S RD
843,12,NaN,NaN,NaN,0,12+ST.+GEORGE'S+RD,ST. GEORGE'S RD
1178,13,NaN,NaN,NaN,0,13+ST.+GEORGE'S+RD,ST. GEORGE'S RD
1431,14,NaN,NaN,NaN,0,14+ST.+GEORGE'S+RD,ST. GEORGE'S RD
1635,15,NaN,NaN,NaN,0,15+ST.+GEORGE'S+RD,ST. GEORGE'S RD
1837,16,NaN,NaN,NaN,0,16+ST.+GEORGE'S+RD,ST. GEORGE'S RD
2048,17,NaN,NaN,NaN,0,17+ST.+GEORGE'S+RD,ST. GEORGE'S RD
2217,18,NaN,NaN,NaN,0,18+ST.+GEORGE'S+RD,ST. GEORGE'S RD


In [33]:
for i in error:
    try:
        record = address_dict[i]
        rec_query = re.sub("[.']", '', record['query'])
        rec_query += ",+Singapore"
        query = 'https://maps.googleapis.com/maps/api/geocode/json?address={0}&key='.format(rec_query)
        resp = requests.get(query)
        if resp.status_code == 200:
            content = resp.json()
            if len(content['results']) > 0:
                coordinates = content['results'][0]['geometry']['location']
                record['coordinates'] = [coordinates['lat'], coordinates['lng']]
                record['found'] = len(content['results'])
                record['full'] = content['results']
                record['ok'] = 1
                continue
        
        print('sth failed at record ' + str(i))
        record['ok'] = 0

    except Exception as e:
        print('exception ' + str(e) + 'at record ' + str(i))
        record['ok'] = 0
    
    time.sleep(0.08)

In [34]:
add = pd.DataFrame(address_dict)
len(add.loc[add['ok'] == 0, :])

0

In [35]:
add.head()

,blk_no,coordinates,found,full,ok,query,street
0,1,"[1.303671351, 103.8644787]",3,"[{'SEARCHVAL': 'BEACH ROAD GARDENS', 'BLK_NO':...",1,1+BEACH+RD,BEACH RD
1,1,"[1.320852087, 103.93372109999999]",2,"[{'SEARCHVAL': 'HDB-BEDOK', 'BLK_NO': '1', 'RO...",1,1+BEDOK+STH+AVE+1,BEDOK STH AVE 1
2,1,"[1.277034744, 103.841261]",3,"[{'SEARCHVAL': 'THE PINNACLE@DUXTON', 'BLK_NO'...",1,1+CANTONMENT+RD,CANTONMENT RD
3,1,"[1.327968792, 103.92271600000001]",1,"[{'SEARCHVAL': 'PING YI GARDENS', 'BLK_NO': '1...",1,1+CHAI+CHEE+RD,CHAI CHEE RD
4,1,"[1.388546562, 103.98780450000001]",2,"[{'SEARCHVAL': 'HDB-CHANGI VILLAGE ROV', 'BLK_...",1,1+CHANGI+VILLAGE+RD,CHANGI VILLAGE RD


In [36]:
pkl.dump(address_dict, open('address_dict_v1.pkl', 'wb'))

## Validation

In [49]:
def get_add(add_list):
    first_rec = add_list[0]
    if 'SEARCHVAL' in first_rec.keys():
        add = "{0}+{1}".format(first_rec['BLK_NO'], first_rec['ROAD_NAME'])
        add = re.sub(' ', '+', add)
        return add
    
    add = "{0}+{1}".format(first_rec['address_components'][0]['long_name'], first_rec['address_components'][2]['long_name'])
    return add

In [50]:
add['first_add'] = [get_add(addresses) for addresses in add['full']]

In [51]:
add.head()

,blk_no,coordinates,found,full,ok,query,street,first_add,diff
0,1,"[1.303671351, 103.8644787]",3,"[{'SEARCHVAL': 'BEACH ROAD GARDENS', 'BLK_NO':...",1,1+BEACH+RD,BEACH RD,1+BEACH+ROAD,0.909091
1,1,"[1.320852087, 103.93372109999999]",2,"[{'SEARCHVAL': 'HDB-BEDOK', 'BLK_NO': '1', 'RO...",1,1+BEDOK+STH+AVE+1,BEDOK STH AVE 1,1+BEDOK+SOUTH+AVENUE+1,0.871795
2,1,"[1.277034744, 103.841261]",3,"[{'SEARCHVAL': 'THE PINNACLE@DUXTON', 'BLK_NO'...",1,1+CANTONMENT+RD,CANTONMENT RD,1+CANTONMENT+ROAD,0.937500
3,1,"[1.327968792, 103.92271600000001]",1,"[{'SEARCHVAL': 'PING YI GARDENS', 'BLK_NO': '1...",1,1+CHAI+CHEE+RD,CHAI CHEE RD,1+CHAI+CHEE+ROAD,0.933333
4,1,"[1.388546562, 103.98780450000001]",2,"[{'SEARCHVAL': 'HDB-CHANGI VILLAGE ROV', 'BLK_...",1,1+CHANGI+VILLAGE+RD,CHANGI VILLAGE RD,1+CHANGI+VILLAGE+ROAD,0.950000


In [41]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


In [52]:
add['diff'] = add.apply(lambda row: similar(row['query'], row['first_add']), axis=1)

In [53]:
add['diff'].describe()

count    12267.000000
mean         0.917533
std          0.050986
min          0.153846
25%          0.900000
50%          0.918919
75%          0.933333
max          1.000000
Name: diff, dtype: float64

In [46]:
error_add = add[add['diff'] < 0.9]

In [56]:
add['full'][2986]

[{'SEARCHVAL': 'BLK 216 AND 215 CHOA CHU KANG CENTRAL',
  'BLK_NO': '',
  'ROAD_NAME': 'NIL',
  'BUILDING': 'BLK 216 AND 215 CHOA CHU KANG CENTRAL',
  'ADDRESS': 'BLK 216 AND 215 CHOA CHU KANG CENTRAL',
  'POSTAL': 'NIL',
  'X': '18402.36455',
  'Y': '40559.98376',
  'LATITUDE': '1.383083024',
  'LONGITUDE': '103.7470766',
  'LONGTITUDE': '103.7470766'},
 {'SEARCHVAL': 'HDB-CHOA CHU KANG',
  'BLK_NO': '215',
  'ROAD_NAME': 'CHOA CHU KANG CENTRAL',
  'BUILDING': 'HDB-CHOA CHU KANG',
  'ADDRESS': '215 CHOA CHU KANG CENTRAL HDB-CHOA CHU KANG SINGAPORE 680215',
  'POSTAL': '680215',
  'X': '18407.291240000002',
  'Y': '40539.04137',
  'LATITUDE': '1.382893631',
  'LONGITUDE': '103.7471209',
  'LONGTITUDE': '103.7471209'},
 {'SEARCHVAL': '215 CHOA CHU KANG CENTRAL',
  'BLK_NO': '215',
  'ROAD_NAME': 'CHOA CHU KANG CENTRAL',
  'BUILDING': 'NIL',
  'ADDRESS': '215 CHOA CHU KANG CENTRAL',
  'POSTAL': 'NIL',
  'X': '18412.05375',
  'Y': '40536.27394',
  'LATITUDE': '1.382868605',
  'LONGITUDE':

In [57]:
add['lat'] = [coord[0] for coord in add['coordinates']]
add['lng'] = [coord[1] for coord in add['coordinates']]

In [58]:
add.head()

,blk_no,coordinates,found,full,ok,query,street,first_add,diff,lat,lng
0,1,"[1.303671351, 103.8644787]",3,"[{'SEARCHVAL': 'BEACH ROAD GARDENS', 'BLK_NO':...",1,1+BEACH+RD,BEACH RD,1+BEACH+ROAD,0.909091,1.303671351,103.8644787
1,1,"[1.320852087, 103.93372109999999]",2,"[{'SEARCHVAL': 'HDB-BEDOK', 'BLK_NO': '1', 'RO...",1,1+BEDOK+STH+AVE+1,BEDOK STH AVE 1,1+BEDOK+SOUTH+AVENUE+1,0.871795,1.320852087,103.93372109999999
2,1,"[1.277034744, 103.841261]",3,"[{'SEARCHVAL': 'THE PINNACLE@DUXTON', 'BLK_NO'...",1,1+CANTONMENT+RD,CANTONMENT RD,1+CANTONMENT+ROAD,0.937500,1.277034744,103.841261
3,1,"[1.327968792, 103.92271600000001]",1,"[{'SEARCHVAL': 'PING YI GARDENS', 'BLK_NO': '1...",1,1+CHAI+CHEE+RD,CHAI CHEE RD,1+CHAI+CHEE+ROAD,0.933333,1.327968792,103.92271600000001
4,1,"[1.388546562, 103.98780450000001]",2,"[{'SEARCHVAL': 'HDB-CHANGI VILLAGE ROV', 'BLK_...",1,1+CHANGI+VILLAGE+RD,CHANGI VILLAGE RD,1+CHANGI+VILLAGE+ROAD,0.950000,1.388546562,103.98780450000001


In [59]:
add_dict_df = add.loc[:, ['blk_no', 'street', 'lat', 'lng']]

In [61]:
print(len(hdb))
hdb_join = hdb.merge(add_dict_df, how='inner', on=['blk_no', 'street'], validate='one_to_one')
print(len(hdb_join))

12267
12267


In [62]:
hdb_join.head()

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,...,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,lat,lng
0,1,BEACH RD,16,1970,Y,Y,N,N,N,N,...,2,0,0,0,0,0,0,0,1.303671351,103.8644787
1,1,BEDOK STH AVE 1,14,1975,Y,N,N,Y,N,N,...,2,0,0,0,0,0,0,0,1.320852087,103.93372109999999
2,1,CANTONMENT RD,2,2010,N,Y,N,N,N,N,...,0,0,0,0,0,0,0,0,1.277034744,103.841261
3,1,CHAI CHEE RD,15,1982,Y,N,N,N,N,N,...,92,0,0,0,0,0,0,0,1.327968792,103.92271600000001
4,1,CHANGI VILLAGE RD,4,1975,Y,Y,N,N,N,N,...,1,0,0,0,0,0,0,0,1.388546562,103.98780450000001


In [63]:
hdb_join.to_csv('hdb-properties-geocoded.csv', index=False)